In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp

def sech(x):
    return 1 / np.cosh(x)

def tanh(x):
    return np.sinh(x) / np.cosh(x)


beta=1
D1 = 0.1
D2 = 0.1
m=1
tspan = np.arange(0, 4.5, 0.5)

Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny

x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)
u=tanh(np.sqrt(X**2+Y**2)) * np.cos(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2))) 
v=tanh(np.sqrt(X**2+Y**2)) * np.sin(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2)))

kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# Define the ODE system
def uv_rhs(t, uv, beta, nx, ny, D1, D2):
    u, v = np.split(uv, 2, axis=0)
    ut = u.reshape(nx, ny)
    vt = v.reshape(nx, ny)
    u = ifft2(ut)
    v = ifft2(vt)
    A2 = u**2 + v**2
    u_rhs = fft2((1-A2)*u - (-beta*A2)*v) - D1*K*ut
    v_rhs = fft2((-beta*A2)*u + (1-A2)*v) - D2*K*vt
    return np.hstack([u_rhs.flatten(), v_rhs.flatten()])

# Solve the ODE and plot the results
uv0 = np.hstack([fft2(u).flatten(), fft2(v).flatten()])
wtsol = solve_ivp(uv_rhs, (tspan[0], tspan[-1]), uv0, method = 'RK45', t_eval=tspan, args=(beta, nx, ny, D1, D2))

A1 = wtsol.y
print(A1.shape)
print(A1)


(8192, 9)
[[ 24.94003847+0.00000000e+00j  12.73268299+4.93789830e-16j
   -1.38095598-2.52640555e-16j ... -64.02389647-1.65360865e-14j
  -67.76356741-2.02432797e-14j -61.18058974-1.79105180e-14j]
 [-18.55666362-5.81663109e+01j -42.51586944-4.69129224e+01j
  -60.80795253-2.57480390e+01j ... -26.39439597+1.13082890e+02j
    6.86544434+1.23000456e+02j  41.4436393 +1.10055312e+02j]
 [-16.04755868+3.28279829e+01j -22.03971648-4.57977740e+01j
  -23.23089505-1.04141716e+02j ... -25.03391682-9.26527314e+01j
  -29.2936105 -4.09594873e+01j -31.3712619 +1.56986891e+01j]
 ...
 [ 24.73021466-5.66774723e+02j  34.94179045-3.31372917e+02j
   38.82924248-4.97842318e+01j ...   4.99619196+6.02396295e+02j
   -9.93322885+4.90736906e+02j -25.6299042 +2.81792021e+02j]
 [ 25.33720124-3.61633792e+02j  43.00958768-4.53711746e+02j
   51.93221654-4.47841562e+02j ... -30.76392977+2.66442187e+02j
  -58.45411318+4.29165358e+02j -74.0191717 +5.05315322e+02j]
 [ -6.4753501 +3.96245454e+01j  15.86720969-5.83358549e+01j


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import kron
from scipy.integrate import odeint
from numpy import *

def sech(x):
    return 1 / np.cosh(x)

def tanh(x):
    return np.sinh(x) / np.cosh(x)

def cheb(N):
	if N==0: 
		D = 0.; x = 1.
	else:
		n = np.arange(0,N+1)
		x = np.cos(np.pi*n/N).reshape(N+1,1) 
		c = (np.hstack(( [2.], np.ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
		X = np.tile(x,(1,N+1))
		dX = X - X.T
		D = np.dot(c,1./c.T)/(dX+np.eye(N+1))
		D -= np.diag(np.sum(D.T,axis=0))
	return D, x.reshape(N+1)

N=30
D, x = cheb(N)
x = x * 10
D[N, :] = 0
D[0, :] = 0
#print(x.shape)
Dsquared = np.dot(D, D) / 100
y = x

I = np.eye(len(Dsquared))
L = kron(I, Dsquared) + kron(Dsquared, I)  # 2D Laplacian
#print(L.shape)
X, Y = np.meshgrid(x, y)

u=tanh(np.sqrt(X**2+Y**2)) * np.cos(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2))) 
v=tanh(np.sqrt(X**2+Y**2)) * np.sin(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2)))

def cheb_rhs(t, uv, beta, L, D1, D2):
    u, v = np.split(uv, 2, axis=0)
    A2 = u**2 + v**2
    u_rhs = (1 - A2)*u - (-beta*A2)*v + D1*np.dot(L, u)
    v_rhs = (-beta*A2)*u + (1 - A2)*v + D2*np.dot(L, v)
    return np.hstack([u_rhs.flatten(), v_rhs.flatten()])

# Solve the ODE and plot the results
uv0 = np.hstack([u.flatten(), v.flatten()])
wtsol = solve_ivp(cheb_rhs, (tspan[0], tspan[-1]), uv0, method = 'RK45', t_eval=tspan, args=(beta, L, D1, D2))

A2 = wtsol.y
print(A2.shape)
print(A2)



(1922, 9)
[[ 0.70358468  0.27678435 -0.21775865 ... -0.79689015 -0.40972859
   0.07776933]
 [ 0.73241275  0.47188952  0.07344742 ... -0.96577657 -0.78500366
  -0.4261521 ]
 [ 0.81058026  0.37605887 -0.11123233 ... -0.84008598 -0.49565779
  -0.03085913]
 ...
 [ 0.58562756  0.91352592  0.97914313 ... -0.50294695 -0.84298442
  -0.97634716]
 [ 0.6808609   0.87018536  0.97997159 ... -0.16453512 -0.5878894
  -0.88455009]
 [ 0.71061143  0.96093661  0.97601586 ... -0.60413504 -0.91222169
  -0.99697897]]
